T6DQ1

What are some common assumptions that underlie hypothesis testing for paired data, multiple population means, and variance comparisons? Discuss the potential consequences of violating these assumptions and suggest possible remedies. Additionally, provide examples of situations where each of these statistical tests may be appropriate, and explain why. Finally, what are some practical limitations or considerations when interpreting the results of these tests? How can Python be used to develop a useful tool in this context?

Resource: 
CreditCardUsage. (n.d.). Www.kaggle.com. Retrieved March 22, 2024, from https://www.kaggle.com/datasets/singhbirender/creditcardusage/data
‌

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [7]:
df = pd.read_csv('cust_seg.csv')
df.head()

,custid,sex,AqChannel,region,Marital_status,segment,pre_usage,Post_usage_1month,Latest_mon_usage,post_usage_2ndmonth
0,70,0,4,1,1,1,57,52,49.2,57.2
1,121,1,4,2,1,3,68,59,63.6,64.9
2,86,0,4,3,1,1,44,33,64.8,36.3
3,141,0,4,3,1,3,63,44,56.4,48.4
4,172,0,4,2,1,2,47,52,68.4,57.2


One sample t-test

In [9]:
stats.ttest_1samp(a = df.Latest_mon_usage, popmean = 50)

TtestResult(statistic=16.57233752433133, pvalue=2.4963719280931942e-39, df=199)

In [10]:
df.Latest_mon_usage.mean()

63.174

Two Sample T-test (Paried)

In [12]:
print(df.pre_usage.mean())
print(df.Post_usage_1month.mean())
print(df.post_usage_2ndmonth.mean())

52.23
52.775
58.0525


In [13]:
stats.ttest_rel(a = df.pre_usage, b = df.Post_usage_1month)

TtestResult(statistic=-0.8673065458794775, pvalue=0.3868186820914985, df=199)

p-value is less than 0.5 so we fail to reject the null hypothesis.

Two Sample T-test (Independent)

In [16]:
Males_spend = df.Post_usage_1month[df.sex == 0]
Females_spend = df.Post_usage_1month[df.sex == 1]

In [17]:
print(Males_spend.head())
print(Females_spend.head())

0    52
2    33
3    44
4    52
5    52
Name: Post_usage_1month, dtype: int64
1     59
92    62
93    44
94    44
95    62
Name: Post_usage_1month, dtype: int64


In [18]:
print(Males_spend.mean())
print(Females_spend.mean())

50.120879120879124
54.99082568807339


In [19]:
print(Males_spend.std())
print(Females_spend.std())

10.305160697259263
8.13371516959346


In [20]:
stats.ttest_ind(a = Males_spend, b = Females_spend, equal_var = False)

TtestResult(statistic=-3.656408047887528, pvalue=0.0003408849359426619, df=169.7073800375754)

In [21]:
# we can use ANOVA as well.
stats.f_oneway(Males_spend, Females_spend)

F_onewayResult(statistic=13.94330754080599, pvalue=0.0002462546120354903)

p-value if less than 0.05 we reject the null hypothesis, hence there is a differnce betwee how much a male and a female spends

ANOVA Test

In [22]:
df.segment.value_counts()

segment
2    105
3     50
1     45
Name: count, dtype: int64

In [24]:
s1 = df.Latest_mon_usage[df.segment == 1]
s2 = df.Latest_mon_usage[df.segment == 2]
s3 = df.Latest_mon_usage[df.segment == 3]

# perform ANOVA test
stats.f_oneway(s1, s2, s3)

F_onewayResult(statistic=29.279283801321778, pvalue=7.36401083352674e-12)

pvalues is less than 0.05. we reject the null hypothesis which means there is a siginficant difference betweeen the average of the segments.

if fvalue is high that means the segments are differnt and when the segments are different that means there is a realtionship between variables

In [25]:
print(s1.mean(), s2.mean(), s3.mean() )

60.02666666666667 68.08 55.70399999999999


ChiSquare Test

In [26]:
t = pd.crosstab(df.segment, df.region, margins = True)
t
# actual distribution between segment and region

region,1,2,3,All
segment,,,,
1,16,20,9,45
2,19,44,42,105
3,12,31,7,50
All,47,95,58,200


In [27]:
stats.chi2_contingency(observed = t)

Chi2ContingencyResult(statistic=16.60444164948934, pvalue=0.055282939487992365, dof=9, expected_freq=array([[ 10.575,  21.375,  13.05 ,  45.   ],
       [ 24.675,  49.875,  30.45 , 105.   ],
       [ 11.75 ,  23.75 ,  14.5  ,  50.   ],
       [ 47.   ,  95.   ,  58.   , 200.   ]]))

Based on the p-value we can say thereis a realtionshion between region and segment

Correlation

In [28]:
print(np.corrcoef(df.Latest_mon_usage, df.Post_usage_1month))

[[1.         0.61744926]
 [0.61744926 1.        ]]


In [29]:
print(stats.stats.pearsonr(df.Latest_mon_usage, df.Post_usage_1month))

PearsonRResult(statistic=0.6174492644854912, pvalue=2.0866647416873645e-22)


/var/folders/x1/3h80s1d10bx2rgftf2cxmc980000gn/T/ipykernel_94473/2688234696.py:1: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  print(stats.stats.pearsonr(df.Latest_mon_usage, df.Post_usage_1month))


Since pval is less than 0.05 we reject the NULL hypothesis and there is a relationship

Correlation talks about the linear relationship between the variables. That means if one is changing the other is also changing.